In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv(".env")
llm_large = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=1.0
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0
)

llm_small = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=1.0
)

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(".env")

llm = ChatOpenAI(
    model="gpt-5",
)

response = llm.invoke("hi there! i am daehan kim")

from util.token_calc import calculate_price_of_messages
from util.testhelper import print_messages
calculate_price_of_messages([response], debug=True)
print_messages([response])

[단일 메시지] 입력 토큰 비용: 0.00002, 출력 토큰 비용: 0.00178, 총비용: 0.00180

📨 총 1개의 메시지

index: 1,
type": AI
{
   "content": "Hi Daehan Kim—nice to meet you! How can I help today? If you have a preferred name or focus area (planning, writing, coding, brainstorming, explanations), let me know.",
   "metadata": {
      "content": "Hi Daehan Kim—nice to meet you! How can I help today? If you have a preferred name or focus area (planning, writing, coding, brainstorming, explanations), let me know.",
      "additional_kwargs": {
         "refusal": null
      },
      "response_metadata": {
         "token_usage": {
            "completion_tokens": 178,
            "prompt_tokens": 15,
            "total_tokens": 193,
            "completion_tokens_details": {
               "accepted_prediction_tokens": 0,
               "audio_tokens": 0,
               "reasoning_tokens": 128,
               "rejected_prediction_tokens": 0
            },
            "prompt_tokens_details": {
               "audio_toke

In [2]:
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv

load_dotenv(".env")

llm = ChatAnthropic(
    model_name="claude-sonnet-4-5",
    timeout=None,
    stop=None
)

response = llm.invoke("hi there! i am daehan kim")

from util.token_calc import calculate_price_of_messages
from util.testhelper import print_messages
calculate_price_of_messages([response], debug=True)
print_messages([response])

KeyError: 'claude-sonnet-4-5-20250929 : 정의된 모델이름이 아닙니다!'

In [2]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage

agent = create_agent(
    model=llm
)

agent.invoke({"messages": [HumanMessage(content="hello!")]})

{'messages': [HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}, id='5a9ed5b6-7156-4e73-8a0b-4c79604d5fb3'),
  AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--000172f8-91d7-41bb-9437-b7498e82ab37-0', usage_metadata={'input_tokens': 3, 'output_tokens': 48, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 38}})]}

In [14]:
from langchain.tools import tool

@tool
def tool_1(query: str) -> str:
    """여행 계획의 개요를 작성하는 도구입니다.
    
    Args:
        query: str = 여행지와 여행 목적
    Return:
        content: str = 여행 계획 개요"""
    
    response = llm_small.invoke(f"다음의 여행지와 여행 목적에 맞도록 여행 계획 개요 항목을 작성해 주세요.\n{query}")

    return str(response.content)

@tool
def tool_2(query: str) -> str:
    """여행 계획 개요 항목 하나에 대한 내용을 작성하는 도구입니다.
    
    Args:
        query: str = 여행지와 여행 목적
    Return:
        content: str = 내용 콘텐츠"""
    response = llm_small.invoke(f"다음의 여행 계획 개요 항목에 대한 콘텐츠를 생성해 주세요.\n{query}")

    return str(response.content)

tools = [tool_1, tool_2]

In [10]:
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """툴의 execution error를 커스텀하여 메시지를 리턴한다."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

In [17]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """request의 값에 따라서 모델을 동적으로 선택할 수 있다."""
    message_count = len(request.messages)

    if message_count > 2:
        model = llm
    else:
        model = llm_small
    
    request.model = model
    return handler(request)

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="당신은 여행 계획 플래너입니다. 도구를 활용하여 사용자의 요구에 대한 계획을 수립하세요. 재질문하지 말고, 최대한 질문의 의도를 파악하고 업무를 수행하세요." \
    "여행 개요와 세부 내용을 작성하세요.",
    middleware=[dynamic_model_selection, handle_tool_errors]
)

for chunk in agent.stream({"messages": [HumanMessage(
    content="안녕하세요. 제주도로 여행 계획을 세워 주세요. 부모님을 모시고 효도여행을 하려고 합니다."
)]}, stream_mode="values"):
    latest_message = chunk["messages"][-1]
    print("="*30)
    if latest_message.content:
        print(f"Agent: {latest_message.content[:100]}...")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: 안녕하세요. 제주도로 여행 계획을 세워 주세요. 부모님을 모시고 효도여행을 하려고 합니다....
Calling tools: ['tool_1']
Agent: ## 제주도 효도 여행 계획 개요

**여행 목적:** 부모님께 편안하고 즐거운 추억을 선사하며, 제주도의 아름다운 자연과 맛있는 음식을 함께 만끽하는 효도 여행

---

###...
Calling tools: ['tool_2', 'tool_2', 'tool_2']
Agent: ## 제주도 효도여행 식사 가이드: 부모님과 함께 즐기는 맛있는 추억

제주도 효도여행에서 식사는 빼놓을 수 없는 중요한 부분이죠! 부모님의 입맛과 건강을 고려하면서도 제주의 다채...
Agent: ## 제주도 효도 여행 계획 개요

**여행 목적:** 부모님께 편안하고 즐거운 추억을 선사하며, 제주도의 아름다운 자연과 맛있는 음식을 함께 만끽하는 효도 여행

---

###...


In [ ]:
from util.testhelper import print_messages
print_messages(response["messages"], mode='content')


📨 총 8개의 메시지

[1] HUMANMESSAGE
안녕하세요. 제주도로 여행 계획을 세워 주세요. 부모님을 모시고 효도여행을 하려고 합니다.
--------------------------------------------------------------------------------

[2] AIMESSAGE

--------------------------------------------------------------------------------

[3] TOOLMESSAGE
## 제주도 효도여행 여행 계획 개요 (안)

**여행지:** 제주도

**여행 목적:** 부모님께 편안하고 즐거운 휴식을 제공하며, 아름다운 자연과 맛있는 음식을 함께 경험하는 효도여행

---

**I. 여행 준비 (출발 전)**

*   **A. 대상자 파악 및 의견 수렴:**
    *   1. 부모님의 선호하는 여행 스타일 (액티비티, 휴양, 관광, 미식 등) 확인
    *   2. 특별히 가고 싶어 하시는 장소나 먹고 싶은 음식 파악
    *   3. 동행 인원 및 각자의 건강 상태 고려
*   **B. 여행 시기 및 기간 결정:**
    *   1. 부모님께서 편안하게 여행하기 좋은 계절 선택 (성수기 피하기 등)
    *   2. 휴식과 여유를 충분히 즐길 수 있는 기간 설정 (무리한 일정 지양)
*   **C. 예산 계획:**
    *   1. 교통비 (항공권, 렌터카 등)
    *   2. 숙박비
    *   3. 식비
    *   4. 관광지 입장료 및 체험비
    *   5. 기타 (기념품, 비상금 등)
*   **D. 예약 및 준비:**
    *   1. **항공권/교통편 예약:** 부모님께서 편안하게 이동할 수 있는 시간대와 좌석 고려
    *   2. **숙소 예약:**
        *   a. 접근성 좋은 위치 (관광지, 병원 등)
        *   b. 편안하고 쾌적한 시설 (엘리베이터, 넓은 욕실 등)
        